# WebsocketClientWorker Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled `WebsocketServerWorker.ipynb` and is in the same folder as this notebook. You should execute this notebook AFTER you have executed the other one.

In this tutorial, we'll demonstrate how to launch a SocketWorker server which will listen for PyTorch commands over a socket connection. In this tutorial, the two workers are connected via a socket connection on the localhost network.

If you'd prefer to download this notebook and run it locally, you can do so via the `Download .ipynb` in the `File` dropdown field in this Google colab environment. 


# Step -1: Copy This Notebook

Go up to File -> Save A Copy in Drive

This will let you execute the notebook (it won't let you execute this one by default)

# Step 0: Install Dependencies

In [ ]:
! git clone https://github.com/OpenMined/PySyft.git
! git checkout torch_031
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

!cd PySyft; pip install -r requirements.txt; python setup.py install

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Step 1: Hook Torch and Create Local Worker

In this step, we hook PyTorch and initialize within the hook a client SocketWorker.

In [ ]:
import syft as sy
from syft.workers.websocket_server import WebsocketServerWorker

hook = sy.TorchHook(torch)

local_worker = WebsocketServerWorker(
                            host="localhost",
                            hook=hook,
                            id=0,
                            port=8182,
                            log_msgs=True,
                            verbose=True)

local_worker.start()  # Might need to Interupt with `control-C`

hook = sy.TorchHook(torch, local_worker=local_worker)

# Step 2: Create Pointer to Remote Socket Worker

In order to interact with a foreign worker over a socket connection, we need to create a pointer to it containing information on how to contact it. We set the is_pointer=True to signify that this Python object is not in fact a worker in and of itself but that it is merely a pointer to one over the network. We then inform our local worker about this pointer.

In [ ]:
from syft.workers.websocket_client import WebsocketClientWorker

remote_client = WebsocketClientWorker(
                            host="localhost",
                            hook=hook,
                            id=2, 
                            port=8182)

hook.local_worker.add_worker(remote_client)

# Step 3: Create Tensors & Send To The Worker

In [ ]:
x = sy.FloatTensor([1,2,3,4,5]).send(remote_client)

In [ ]:
x2 = torch.FloatTensor([1,2,3,4,4]).send(remote_client)

In [ ]:
x

In [ ]:
x2

# Step 4: Execute Operations Like Normal

In [ ]:
y = x + x2 + x

# Step 5: Get Results

In [ ]:
y

In [ ]:
y.get()

In [ ]:
y